In [1]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import *
from traitlets import *

import numpy as np
import pandas as pd
import bqplot as bq

In [2]:
from bqplot import *

In [3]:
from bqplot import pyplot as plt

In [4]:
from traitlets import dlink

In [5]:
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)

In [6]:
from ipywidgets import Label, ToggleButtons, VBox

In [7]:

# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [78]:
wind_radii=np.array([34.,50.,64.])*kt2ms

In [128]:
def hvel(r,dic):
    for key,val in dic.items():
        exec(key + '=val') 
    x=0.5+(r-rmax)/(maxR-rmax)*k
    return (b/rhoa*(rmax/r)**b*dph*np.exp(-(rmax/r)**b))**x

In [129]:
rh=np.linspace(0.01,maxR,100)

### read TC data

In [152]:
tc=pd.read_csv('test/step3.txt',delimiter='\t')

In [153]:
tc

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,w34sw,w34nw,vmax0vt,b,dph,k,np.max(V),rmaxh,rmse,vmax1
0,0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,...,0.000000,0.000000,5.069814,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,...,18.059481,16.718113,18.059481,0.814397,1288.689225,0.235803,18.059481,21236.795240,0.832487,18.322971
2,12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,...,18.366306,17.301589,18.366306,0.844967,1291.038196,0.238350,18.366306,22989.998120,0.705101,18.680701
3,18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,...,18.525193,17.452947,25.859722,1.076894,1981.089735,0.234723,25.859722,17570.521694,2.037823,26.124160
4,24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,...,18.632523,17.383112,25.728973,1.158931,1829.735742,0.223864,25.728973,19772.107096,1.984706,26.045152
5,30.0,2010-10-30 12:00:00,TOMAS,13.1,-60.1,993,1009,37040.0,33.438889,222240.0,...,18.645319,17.719449,30.031688,1.150231,2495.034488,0.248659,30.031688,16268.555966,2.391121,30.299449
6,36.0,2010-10-30 18:00:00,TOMAS,13.3,-61.0,990,1009,37040.0,41.155556,222240.0,...,18.630080,17.783646,33.720308,1.060996,3386.729585,0.236639,30.216264,10990.778052,2.444031,33.904005
7,42.0,2010-10-31 00:00:00,TOMAS,13.5,-61.7,982,1009,37040.0,43.727778,222240.0,...,18.658875,17.847466,36.492381,0.976127,4303.478427,0.159327,32.008721,9746.760819,2.971324,36.657827
8,48.0,2010-10-31 06:00:00,TOMAS,13.8,-62.4,983,1009,27780.0,43.727778,222240.0,...,18.727010,17.858079,36.023263,0.988988,4141.195562,0.191431,32.136380,9971.037054,3.155295,36.196141
9,54.0,2010-10-31 12:00:00,TOMAS,14.0,-63.3,988,1011,27780.0,41.155556,222240.0,...,18.784244,17.817282,32.757192,1.072962,3172.902554,0.213743,32.522233,13874.571487,3.043061,33.000800


In [154]:
i=10

In [156]:
tc.ix[i:i,['dph','b','rmax','k']].T.to_dict()[i]


{'b': 1.0398479901270798,
 'dph': 3288.183929325558,
 'k': 0.2423797625633039,
 'rmax': 27780.0}

In [155]:
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

ax_x = bq.Axis(label='X Axis', scale=x_sc, grid_lines='solid')
ax_y = bq.Axis(label='Y Axis', scale=y_sc, orientation='vertical', grid_lines='solid')

prop = tc.ix[i:i,['dph','b','rmax','k']].T.to_dict()[i]

p64=tc.iloc[i][['64ne','64se', '64sw', '64nw']].astype(float)

p50=tc.iloc[i][['50ne','50se', '50sw', '50nw']].astype(float)

p34=tc.iloc[i][['34ne','34se', '34sw', '34nw']].astype(float)

vmax=tc.iloc[i].vmax

x_data=np.concatenate([p34.values,p50.values,p64.values])

x_data[x_data==0.]=np.nan

y_data=np.concatenate([np.ones(4)*wind_radii[0],np.ones(4)*wind_radii[1],np.ones(4)*wind_radii[2]])

y_data[np.isnan(x_data)]=np.nan

y_data=y_data[~np.isnan(y_data)]

x_data=x_data[~np.isnan(x_data)]


Line = bq.Lines(x=rh,
                     y=hvel(rh, prop),
                     animate_dur=1000,
                     scales={'x': x_sc, 'y': y_sc})


LineVmax = bq.Lines(x=rh,
                     y=np.ones(rh.size)*vmax,
                     animate_dur=1000,
                     scales={'x': x_sc, 'y': y_sc},colors=['black'], line_style = 'dashed' )



scatter_chart = Scatter(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc},default_colors=['red'])
scatter_chart.enable_move=True


slider1 = IntSlider(value=0, min=0 , max=tc.shape[0]-1 , step=1, description='time')

# Wiring the function parameter with a slider
def select_time(change):
    new = change['new']
    prop = tc.ix[new:new,['dph','b','rmax','k']].T.to_dict()[new]
    p64=tc.iloc[new][['64ne','64se', '64sw', '64nw']].astype(float)
    p50=tc.iloc[new][['50ne','50se', '50sw', '50nw']].astype(float)
    p34=tc.iloc[new][['34ne','34se', '34sw', '34nw']].astype(float)
    vmax=tc.iloc[new].vmax
    x_data=np.concatenate([p34.values,p50.values,p64.values])
    x_data[x_data==0.]=np.nan
    y_data=np.concatenate([np.ones(4)*wind_radii[0],np.ones(4)*wind_radii[1],np.ones(4)*wind_radii[2]])
    y_data[np.isnan(x_data)]=np.nan
    y_data=y_data[~np.isnan(y_data)]
    x_data=x_data[~np.isnan(x_data)]
    Line.y = hvel(rh, prop)
    LineVmax.y = np.ones(rh.size)*vmax
    scatter_chart.x = x_data
    scatter_chart.y = y_data


select_time({'new': i})
slider1.observe(select_time, names=['value'])


fig = bq.Figure(axes=[ax_x, ax_y], marks=[Line, scatter_chart, LineVmax], title_style={'font-size': '20px','fill': 'DarkOrange'}, title='')


#compute rmse
#rmse=hvel(x_data, slider1.value, slider2.value, slider3.value, slider4.value)-y_data
#rmse_label.text = [str(rmse.sum())]
    

VBox([fig, slider1])



In [70]:
tc.ix[1:1,['deltap','b','rmax','k']].T.to_dict()[1]

{'b': 0.81439656511951353,
 'deltap': 1288.6892252911725,
 'k': 0.23580308692366039,
 'rmax': 21236.795240118998}

In [81]:
tc.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,w34sw,w34nw,vmax0vt,b,dph,k,np.max(V),rmaxh,rmse,vmax1
0,0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,...,0.000000,0.000000,5.069814,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,...,18.059481,16.718113,18.059481,0.814397,1288.689225,0.235803,18.059481,21236.795240,0.832487,18.322971
2,12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,...,18.366306,17.301589,18.366306,0.844967,1291.038196,0.238350,18.366306,22989.998120,0.705101,18.680701
3,18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,...,18.525193,17.452947,25.859722,1.076894,1981.089735,0.234723,25.859722,17570.521694,2.037823,26.124160
4,24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,...,18.632523,17.383112,25.728973,1.158931,1829.735742,0.223864,25.728973,19772.107096,1.984706,26.045152
